In [1]:
# Prediksjonsvariabler
import arcpy
from arcpy import env
from arcpy.sa import*
import math
import os
import numpy as np

print(os.getcwd())
arcpy.CheckOutExtension("Spatial")

env.overwriteOutput = True
env.workspace = "Q:\\Project\\Predictive Variables\\Predictive Variables.gdb"

c:\Masteroppgave\Script


Prediksjonsvariabler - 10.11.23

In [2]:
dtm = "Q:\\5m External Validation Data\\elevation.tif"

In [4]:
# Slope degrees

with arcpy.EnvManager(snapRaster=dtm, mask=dtm):
    out_degrees_raster = arcpy.sa.Slope(
        in_raster=dtm,
        output_measurement="DEGREE",
        z_factor=1,
        method="PLANAR",
        z_unit="METER",
        analysis_target_device="GPU_THEN_CPU"
    )
out_degrees_raster.save("Q:\\5m External Validation Data\\Slope_Degrees.tif")

slope_degrees = out_degrees_raster

with arcpy.EnvManager(snapRaster=dtm, mask=dtm):
    out_percent_raster = arcpy.sa.Slope(
        in_raster=dtm,
        output_measurement="PERCENT_RISE",
        z_factor=1,
        method="PLANAR",
        z_unit="METER",
        analysis_target_device="GPU_THEN_CPU"
    )
out_percent_raster.save("Q:\\5m External Validation Data\\Slope_Percent.tif")

slope_percent = out_percent_raster

# Slope radians:
slope_rad = "Q:\\5m External Validation Data\\Slope_Radians.tif"
slope_in_radians = Raster(slope_degrees)
rrconvert = (slope_in_radians * 3.141593 / 180) 
rrconvert.save(slope_rad)
print("Calculation of slope raster in radians completed. saved to" + slope_rad)


Calculation of slope raster in radians completed. saved toQ:\5m External Validation Data\Slope_Radians.tif


# Topographic Wetness Index (TWI)

In [ ]:
# Topographic Wetnes Index (TWI)


with arcpy.EnvManager(parallelProcessingFactor="100%"):
    out_surface_raster = arcpy.sa.Fill(
        in_surface_raster=dtm,
        z_limit=None
    )

    
out_surface_raster.save("Q:\\5m External Validation Data\\HydroDTM.tif")
hydrodtm = out_surface_raster
print(hydrodtm)


with arcpy.EnvManager(snapRaster=dtm, parallelProcessingFactor="100%"):
    out_flow_direction_raster = arcpy.sa.FlowDirection(
        in_surface_raster=hydrodtm,
        force_flow="NORMAL",
        out_drop_raster=None,
        flow_direction_type="D8"
    )
out_flow_direction_raster.save("Q:\\5m External Validation Data\\FlowDir_asf.tif")
flowdir = out_flow_direction_raster
print(flowdir)


with arcpy.EnvManager(snapRaster=dtm, parallelProcessingFactor="100%"):
    out_accumulation_raster = arcpy.sa.FlowAccumulation(
        in_flow_direction_raster=flowdir,
        in_weight_raster=None,
        data_type="FLOAT",
        flow_direction_type="D8"
    )
out_accumulation_raster.save("Q:\\5m External Validation Data\\FlowAcc.tif")

flow_acc = out_accumulation_raster


flow_acc = "Q:\\5m External Validation Data\\FlowAcc.tif"
slope_percent = "Q:\\5m External Validation Data\\Slope_Percent.tif"


# Calculate the TWI
cellsize_x = float(arcpy.management.GetRasterProperties(flow_acc,"CELLSIZEX").getOutput(0).replace(",", "."))
cellsize_y = float(arcpy.management.GetRasterProperties(flow_acc,"CELLSIZEY").getOutput(0).replace(",", "."))
print(cellsize_x, cellsize_y)

expression = f"Ln((('{flow_acc}'+1)*({cellsize_x}*{cellsize_y}))/(('{slope_percent}'/100)+0.0001))"
output_twi_raster = "Q:\\5m External Validation Data\\twi.tif"
arcpy.gp.RasterCalculator_sa(expression, output_twi_raster)
# ________________________________________________________________________


# TPI (Topographic Position Index)

The calculation of TPI at small and large scale is intended for use to calculate landforms

In [3]:
# Topographic position index (TPI)
dtm = ("Q:\\5m Training Data Rasters\\elevation.tif")
arcpy.env.workspace = ('Q:\\5m Training Data Rasters\\')
arcpy.env.overwriteOutput = True


# TPI 2000 - At his scale the major ridge lines and drainages are highlighted.Smaller lateral features disappear
with arcpy.EnvManager(snapRaster=dtm, mask=dtm):
    out_raster = arcpy.ia.FocalStatistics(
        in_raster=dtm,
        neighborhood="Annulus 370 400 CELL", # With a cell size of 5 meters, this equates to a scale factor of 2000
        statistics_type="MEAN",
        ignore_nodata="DATA",
        percentile_value=90
    )
    out_raster.save("TPI_Focal_2000_5m.tif")

arcpy.ddd.Minus(
    in_raster_or_constant1=dtm,
    in_raster_or_constant2="TPI_Focal_2000_5m.tif",
    out_raster="tpi2000_5m_new.tif"
)


# TPI 300 - At this scale a dendridicnetwork of both main and lateral ridges and drainages are revealed
with arcpy.EnvManager(snapRaster=dtm, mask=dtm):
    out_raster = arcpy.ia.FocalStatistics(
        in_raster=dtm,
        neighborhood="Annulus 30 60 CELL",
        statistics_type="MEAN",
        ignore_nodata="DATA",
        percentile_value=90
    )
    out_raster.save("DTM_Focal_300_5m.tif")

arcpy.ddd.Minus(
    in_raster_or_constant1=dtm,
    in_raster_or_constant2="DTM_Focal_300_5m.tif",
    out_raster="tpi300_5m_new.tif"
)


<Result 'Q:\\5m Training Data Rasters\\tpi300_5m_new.tif'>

# Landforms

Combining TPI at a small and large scale allows a variety of nested
landforms to be distinguished. The small and large scale TPI is created based on the focal ststistics under (TPI) calculation in the previous code 

NB! The variables tp300_stdi and tp2000_stdi need to be standardized by centering the data around zero by subtracting the mean and dividing by the standard deviation. This can be done using the Raster Calculator tool in ArcGIS Pro

In [4]:
arcpy.env.workspace = ('Q:\\5m Training Data Rasters\\')
tp300_stdi = "tpi300_5m_new_stdi.tif"
tp2000_stdi = "tpi2000_5m_new_stdi.tif"

slope = "Slope_Degrees.tif"

# Perform the conditional operations
conditions = [
    ((Raster(tp300_stdi) > -100) & (Raster(tp300_stdi) < 100) & (Raster(tp2000_stdi) > -100) & (Raster(tp2000_stdi) < 100) & (Raster(slope) <= 5), 5),
    ((Raster(tp300_stdi) > -100) & (Raster(tp300_stdi) < 100) & (Raster(tp2000_stdi) > -100) & (Raster(tp2000_stdi) < 100) & (Raster(slope) >= 6), 6),
    ((Raster(tp300_stdi) > -100) & (Raster(tp300_stdi) < 100) & (Raster(tp2000_stdi) >= 100), 7),
    ((Raster(tp300_stdi) > -100) & (Raster(tp300_stdi) < 100) & (Raster(tp2000_stdi) <= -100), 4),
    ((Raster(tp300_stdi) <= -100) & (Raster(tp2000_stdi) > -100) & (Raster(tp2000_stdi) < 100), 2),
    ((Raster(tp300_stdi) >= 100) & (Raster(tp2000_stdi) > -100) & (Raster(tp2000_stdi) < 100), 9),
    ((Raster(tp300_stdi) <= -100) & (Raster(tp2000_stdi) >= 100), 3),
    ((Raster(tp300_stdi) <= -100) & (Raster(tp2000_stdi) <= -100), 1),
    ((Raster(tp300_stdi) >= 100) & (Raster(tp2000_stdi) >= 100), 10),
    ((Raster(tp300_stdi) >= 100) & (Raster(tp2000_stdi) <= -100), 8)
]

# Initialize the result with a default value
result = Raster(tp300_stdi) * 0

for condition, value in conditions:
    result = Con(condition, value, result)
result.save("landforms_new.tif")
